In [1]:
import pandas as pd

In [10]:
def get_sig_data_dict(data_file):
    
    data_dict = {}
    data_p_dict = {'up': [], 'down': []}
    data_m_dict = {'up': [], 'down': []}
    data_aa_dict = {'up': [], 'down': []}
    
    data_df = pd.read_csv(data_file, sep = "\t", index_col = 0)
    r, c = data_df.shape
    feature_list = data_df.index.values
    
    for i in range(r):
        feature = feature_list[i]
        cohen_d = data_df.iloc[i][0]
        pval = data_df.iloc[i][4]
        # print (feature, pval)

        if feature != "age":
    
            if pval < 0.05:
                if cohen_d > 0.5:
                    data_dict[feature] = [cohen_d, pval]
                    data_m_dict['up'].append(feature)                
    
                if cohen_d < -0.5:
                    data_dict[feature] = [cohen_d, pval]
                    data_m_dict['down'].append(feature)

    return data_dict, data_p_dict, data_m_dict, data_aa_dict
    
def make_venndiagram(low_omics_dict, high_omics_dict, omics_type, updown_status, comparison_type, seta, setb):
    
    if updown_status == "up":
        intersection_list = list(set(low_omics_dict["up"]) & set(high_omics_dict["up"]))
        uniq_in_low_age_list = list(set(low_omics_dict["up"]) - set(high_omics_dict["up"]))
        uniq_in_high_age_list = list(set(high_omics_dict["up"]) - set(low_omics_dict["up"]))
        
    if updown_status == "down":
        intersection_list = list(set(low_omics_dict["down"]) & set(high_omics_dict["down"]))
        uniq_in_low_age_list = list(set(low_omics_dict["down"]) - set(high_omics_dict["down"]))
        uniq_in_high_age_list = list(set(high_omics_dict["down"]) - set(low_omics_dict["down"]))

    output_file = '../../../analysis/age_stratified/differential_abundance_logit/plot/%s.%s.%s.%s.%s.tsv' % (omics_type, comparison_type, updown_status, seta, setb)
    output_txt = open(output_file, 'w')
    
    output_txt.write('Intersection')
    for feature in intersection_list:
          output_txt.write('\t%s' % feature)
    output_txt.write('\n')
          
    output_txt.write('Uniq in %s' % seta)
    for feature in uniq_in_low_age_list:
          output_txt.write('\t%s' % feature)
    output_txt.write('\n')
          
    output_txt.write('uniq in %s' % setb)
    for feature in uniq_in_high_age_list:
          output_txt.write('\t%s' % feature)
    output_txt.write('\n')

    output_txt.close()


In [11]:
data_dir = '../../../analysis/age_stratified/differential_abundance_logit/'
comparison_list = ['negVSpos']

for comparison_type in comparison_list:
    
    high_age_file = '%shigh_age.%s.tsv' % (data_dir, comparison_type)
    med_age_file = '%smid_age.%s.tsv' % (data_dir, comparison_type)
    low_age_file = '%slow_age.%s.tsv' % (data_dir, comparison_type)
    
    high_dict, high_p_dict, high_m_dict, high_aa_dict = get_sig_data_dict(high_age_file)
    low_dict, low_p_dict, low_m_dict, low_aa_dict = get_sig_data_dict(low_age_file)

    #m
    make_venndiagram(low_m_dict, high_m_dict, "metabolite", "up", comparison_type, 'young','old')
    make_venndiagram(low_m_dict, high_m_dict, "metabolite", "down", comparison_type, 'young','old')
    
